In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Einlesen von der Faktentabelle Tageswetterdaten

In [19]:
df = spark.read.csv("gs://dabi-project-bucket/DWH/tageswetterdaten.csv", inferSchema=True, header=True)

In [2]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/wind", inferSchema=True, header=True)
df.createOrReplaceTempView("wind")

In [5]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr,
        MONTH(Datum) as Monat, 
        MAX(Tagesmax_Windspitze) as max_windspitze, 
        MAX(Tagesavg_Windgeschwindigkeit) as max_windspeed 
    FROM wind 
    GROUP BY Jahr, Monat
    """
)

df_batch_result.toPandas()

,Jahr,Monat,max_windspitze,max_windspeed
0,1997,11,49.5,18.3
1,2022,10,27.3,11.8
2,1995,12,36.8,15.0
3,2005,5,30.7,11.6
4,1998,7,36.7,11.2
...,...,...,...,...
355,2015,5,41.8,18.1
356,1996,8,23.6,10.9
357,2007,3,39.7,17.3
358,2021,4,27.9,13.3


In [6]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/DIM_weather/max_Wind", mode="overwrite")

,stations_id,Datum,Taegliche_Niederschlagshoehe,Niederschlagsform,Tageswert_Schneehoehe
0,3631,1994-01-01,0.5,8.0,0.0
1,3631,1994-01-02,4.6,8.0,0.0
2,3631,1994-01-03,3.0,6.0,0.0
3,3631,1994-01-04,1.5,6.0,0.0
4,3631,1994-01-05,3.5,6.0,0.0
...,...,...,...,...,...
151114,850,2023-12-27,0.3,4.0,0.0
151115,850,2023-12-28,0.6,4.0,0.0
151116,850,2023-12-29,13.0,4.0,0.0
151117,850,2023-12-30,0.0,0.0,NaN


In [47]:
df_sunshine = spark.sql("SELECT stations_id, Datum, SDK as taegliche_Sonnenscheindauer, NM as Tagesmittel_Bedeckungsgrad FROM facts")
df_sunshine.toPandas()

,stations_id,Datum,taegliche_Sonnenscheindauer,Tagesmittel_Bedeckungsgrad
0,3631,1994-01-01,0.8,5.3
1,3631,1994-01-02,0.0,7.7
2,3631,1994-01-03,0.1,7.7
3,3631,1994-01-04,0.3,7.7
4,3631,1994-01-05,0.7,7.3
...,...,...,...,...
151114,850,2023-12-27,NaN,NaN
151115,850,2023-12-28,NaN,NaN
151116,850,2023-12-29,NaN,NaN
151117,850,2023-12-30,NaN,NaN


In [25]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/sunshine", inferSchema=True, header=True)
df.createOrReplaceTempView("sunshine")

In [35]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        AVG(taegliche_Sonnenscheindauer) as avg_Sonnenscheindauer,
        AVG(Tagesmittel_Bedeckungsgrad) as avg_Bedeckungsgrad
    FROM 
        sunshine
    GROUP BY 
        Jahr, Monat
    """
)
df_batch_result.toPandas()

,Jahr,Monat,avg_Sonnenscheindauer,avg_Bedeckungsgrad
0,1997,11,2.096970,6.134359
1,2022,10,4.772847,5.848106
2,1995,12,1.415543,6.180376
3,2005,5,6.945161,5.134243
4,1998,7,5.662581,5.724194
...,...,...,...,...
355,2015,5,6.150323,5.630085
356,1996,8,6.611730,4.879156
357,2007,3,5.542294,4.668983
358,2021,4,6.470000,5.100000


In [36]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/DIM_Weather/avg_Sunshine", mode="overwrite")

In [53]:
df_air = spark.sql("SELECT stations_id, Datum, VPM as Tagesmittel_Dampfdruck, NM as Tagesmittel_Bedeckungsgrad, PM FROM facts")
df_air.toPandas()

,stations_id,Datum,Tagesmittel_Dampfdruck,Tagesmittel_Bedeckungsgrad,PM
0,3631,1994-01-01,7.1,5.3,1006.80
1,3631,1994-01-02,6.3,7.7,993.50
2,3631,1994-01-03,8.3,7.7,995.10
3,3631,1994-01-04,8.3,7.7,988.20
4,3631,1994-01-05,8.2,7.3,984.20
...,...,...,...,...,...
151114,850,2023-12-27,7.4,NaN,1010.79
151115,850,2023-12-28,8.5,NaN,1003.00
151116,850,2023-12-29,9.3,NaN,998.88
151117,850,2023-12-30,8.0,NaN,1003.64


In [50]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/air", inferSchema=True, header=True)
df.createOrReplaceTempView("air")

In [51]:
df_batch_result = spark.sql( """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        AVG(Tagesmittel_Dampfdruck) as avg_Dampfdruck,
        
    FROM 
        air
    GROUP BY 
        Jahr, Monat
    """)
df_batch_result.toPandas()

AnalysisException: [UNRESOLVED_COLUMN.WITHOUT_SUGGESTION] A column or function parameter with name `Datum` cannot be resolved. ; line 3 pos 13;
'Aggregate ['Jahr, 'Monat], ['YEAR('Datum) AS Jahr#776, 'MONTH('Datum) AS Monat#777, 'AVG('Tagesmittel_Dampfdruck) AS avg_Dampfdruck#778, 'FROM AS air#779]
+- OneRowRelation


In [52]:
df_air = spark.sql("SELECT stations_id, Datum, Tagesmittel_Dampfdruck, Tagesmittel_Luftdruck, Tagesmittel_RelFeuchte_Hoehe FROM air")
df_air.toPandas()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Tagesmittel_RelFeuchte_Hoehe` cannot be resolved. Did you mean one of the following? [`Tagesmittel_RelFeuchte`, `Tagesmittel_Luftdruck`, `Tagesmittel_Dampfdruck`, `stations_id`, `Datum`].; line 1 pos 74;
'Project [stations_id#766, Datum#767, Tagesmittel_Dampfdruck#768, Tagesmittel_Luftdruck#769, 'Tagesmittel_RelFeuchte_Hoehe]
+- SubqueryAlias air
   +- View (`air`, [stations_id#766,Datum#767,Tagesmittel_Dampfdruck#768,Tagesmittel_Luftdruck#769,Tagesmittel_RelFeuchte#770])
      +- Relation [stations_id#766,Datum#767,Tagesmittel_Dampfdruck#768,Tagesmittel_Luftdruck#769,Tagesmittel_RelFeuchte#770] parquet


In [14]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/temperature", inferSchema=True, header=True)
df.createOrReplaceTempView("temperature")

In [55]:
df_batch_result = spark.sql("SELECT * FROM temperature")

df_batch_result.toPandas()

,stations_id,Datum,Avg_Lufttemperatur_2m_Hoehe,Max_Lufttemperatur_2m_Hoehe,Min_Lufttemperatur_2m_Hoehe,Min_Lufttemperatur_5cm_Hoehe
0,3631,1994-01-01,3.0,4.7,2.5,1.7
1,3631,1994-01-02,0.8,2.9,0.0,0.2
2,3631,1994-01-03,4.6,5.3,0.7,0.5
3,3631,1994-01-04,5.6,6.5,4.1,3.4
4,3631,1994-01-05,4.9,6.3,3.5,2.6
...,...,...,...,...,...,...
151114,850,2023-12-27,4.2,NaN,NaN,NaN
151115,850,2023-12-28,10.1,NaN,NaN,NaN
151116,850,2023-12-29,9.3,NaN,NaN,NaN
151117,850,2023-12-30,7.1,NaN,NaN,NaN


In [21]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        MAX(Min_Lufttemperatur_5cm_Hoehe) as max_Lufttemperatur_Erdboden
    FROM 
        temperature
    GROUP BY 
        Jahr, Monat
    """
)
df_batch_result.toPandas()

,Jahr,Monat,max_Lufttemperatur_Erdboden
0,1997,11,10.4
1,2022,10,14.4
2,1995,12,8.5
3,2005,5,17.9
4,1998,7,21.3
...,...,...,...
355,2015,5,13.3
356,1996,8,17.7
357,2007,3,6.8
358,2021,4,8.7


In [56]:
df_all = spark.sql("SELECT * FROM facts LIMIT 5")
df_all.toPandas()

,_c0,stations_id,Datum,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
0,0,3631,1994-01-01,3.0,16.4,5.8,10.0,0.5,8.0,0.8,0.0,5.3,7.1,1006.8,3.0,94.0,4.7,2.5,1.7,None
1,1,3631,1994-01-02,3.0,12.6,4.4,10.0,4.6,8.0,0.0,0.0,7.7,6.3,993.5,0.8,96.0,2.9,0.0,0.2,None
2,2,3631,1994-01-03,3.0,17.5,4.6,10.0,3.0,6.0,0.1,0.0,7.7,8.3,995.1,4.6,99.0,5.3,0.7,0.5,None
3,3,3631,1994-01-04,3.0,17.4,7.1,10.0,1.5,6.0,0.3,0.0,7.7,8.3,988.2,5.6,92.0,6.5,4.1,3.4,None
4,4,3631,1994-01-05,3.0,15.3,6.2,10.0,3.5,6.0,0.7,0.0,7.3,8.2,984.2,4.9,96.0,6.3,3.5,2.6,None


# Speichern der Dimensionstabelle in Parquet

In [22]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/DIM_weather/max_temperature", mode="overwrite")

In [8]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/rain", inferSchema=True, header=True)
df.createOrReplaceTempView("rain")

In [23]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        MIN(Taegliche_Niederschlagshoehe) as min_Niederschlagshoehe
    FROM 
        rain 
    GROUP BY 
        Jahr, Monat
    """
)

# In Pandas DataFrame umwandeln
df_batch_result.toPandas()

,Jahr,Monat,min_Niederschlagshoehe
0,1997,11,0.0
1,2022,10,0.0
2,1995,12,0.0
3,2005,5,0.0
4,1998,7,0.0
...,...,...,...
355,2015,5,0.0
356,1996,8,0.0
357,2007,3,0.0
358,2021,4,0.0


In [24]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/DIM_weather/min_Rain", mode="overwrite")

In [25]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/air", inferSchema=True, header=True)
df.createOrReplaceTempView("air")

In [27]:
df_batch_result = spark.sql("SELECT * FROM air")

df_batch_result.toPandas()

,stations_id,Datum,Tagesmittel_Dampfdruck,Tagesmittel_Luftdruck,Tagesmittel_RelFeuchte
0,3631,1994-01-01,7.1,1006.80,94.00
1,3631,1994-01-02,6.3,993.50,96.00
2,3631,1994-01-03,8.3,995.10,99.00
3,3631,1994-01-04,8.3,988.20,92.00
4,3631,1994-01-05,8.2,984.20,96.00
...,...,...,...,...,...
151114,850,2023-12-27,7.4,1010.79,89.21
151115,850,2023-12-28,8.5,1003.00,68.58
151116,850,2023-12-29,9.3,998.88,80.67
151117,850,2023-12-30,8.0,1003.64,79.54


In [35]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        AVG(Tagesmittel_Dampfdruck) as avg_Dampfdruck,
        AVG(Tagesmittel_Luftdruck) as avg_luftdruck,
        AVG(Tagesmittel_RelFeuchte) as avg_RelFeuchte
        
    FROM 
        air
    GROUP BY 
        Jahr, Monat
    """
)

# In Pandas DataFrame umwandeln
df_batch_result.toPandas()

,Jahr,Monat,avg_Dampfdruck,avg_luftdruck,avg_RelFeuchte
0,1997,11,7.024359,962.016667,84.346154
1,2022,10,12.088426,975.448530,83.403449
2,1995,12,4.818011,968.379032,85.706989
3,2005,5,10.511521,966.834274,72.576037
4,1998,7,13.890054,963.671774,75.303763
...,...,...,...,...,...
355,2015,5,9.714100,971.923297,70.232104
356,1996,8,14.571464,967.518952,75.816377
357,2007,3,6.958065,970.808602,75.652074
358,2021,4,6.232857,973.808889,68.057143


In [36]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/DIM_weather/avg_air", mode="overwrite")